In questo esempio si utilizzerà il dataset pubblico CIFAR 10

## Requisiti/Prerequisiti 

- Un account Azure con abbonamento attivo
- Azure Machine Learning workspace 
- Python Environment
- Installazione di Azure ML Python SDK Version 2

## Learning 
- Connettersi all'area di lavoro utilizzando Python SDK v2
- Impostazione del comando per scaricare i dati da un URL Web all'archiviazione BLOB dell'area di lavoro AML eseguendo un processo.
- Utilizzare questi dati memorizzati nell'archiviazione BLOB dell'area di lavoro AML come input per il comando train job.
- Formazione distribuita del modello Pytorch.

# 1. Connettersi ad Azure Machine Learning Workspace

## 1.1 Importare le librerie necessarie


In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Connettersi al worspace usando le credenziali di Default di Azure

Le credenziali di Azure permettono di gestire la maggior parte delle autenticazioni

In [ ]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

## 1.3 Gestisci l'area di lavoro e recuperando il cluster di calcolo allegato


In [ ]:
cluster_name = "cpu-cluster"

In [ ]:
ml_client.workspace_name

In [ ]:
workspace = ml_client.workspace_name
subscription_id = ml_client.workspaces.get(workspace).id.split("/")[2]
resource_group = ml_client.workspaces.get(workspace).resource_group

# 2. Comandi di Configurazione e Esecuzione 

In questa sezione configureremo ed eseguiremo due lavori autonomi.
- **Comando** per leggere e scrivere dati
- **Comando** per il lavoro di formazione distribuito.

Il comando consente all'utente di configurare i seguenti aspetti chiave.
- **code** - Questo è il percorso in cui si trova il codice per eseguire il comando
- **comando** - Questo saràl comando eseguito
- **inputs** - Questo è il dizionario degli input che utilizza coppie di valori di nome e comando. La chiave è un nome per l'input nel contesto del processo e il valore è il valore di input. Gli input possono essere referenziati nel comando utilizzando l'espressione **\${{inputs.<input_name>}}** . Per usare file o cartelle come input, possiamo usare la classe Input. La classe Input supporta tre parametri:
    - **type** - Il tipo di input. Questo può essere un uri_file o uri_folder. Il valore predefinito è uri_folder.
    - **path** - Il percorso del file o della cartella. Questi possono essere file o cartelle locali o remoti. Per i file remoti Azure supporta - http/https, wasb 
        - I **dati/dataset** o **l'archivio dati** di Azure ML sono di tipo uri_folder. Per utilizzare i dati/dataset come input, è possibile utilizzare il dataset registrato nell'area di lavoro utilizzando il formato '<nome_data>:'. Ad esempio Input(type='uri_folder', path='my_dataset:1')
    - **mode** - Modalità di come i dati devono essere consegnati alla destinazione di calcolo. I valori consentiti sono ro_mount, rw_mount e download. Il valore predefinito è ro_mount
- **environment** - Indica l'ambiente dove sarà eseguito il comando. È possibile utilizzare ambienti curati o personalizzati dall'area di lavoro. Oppure può essere creato e utilizzato anche un ambiente personalizzato.
- **compute** - Il calcolo su cui verrà eseguito il comando. In questo esempio viene utilizzato un calcolo chiamato **cpu-cluster** presente nell'area di lavoro. È possibile sostituirlo in qualsiasi altro calcolo nell'area di lavoro. Lo si può eseguire sulla macchina **locale** usando **local** per il calcolo. Questo eseguirà il comando sul computer locale e tutti i dettagli di esecuzione e l'output del processo verranno caricati nell'area di lavoro di Azure ML.
- **distribution** - Configurazione di distribuzione per scenari di formazione distribuiti. Azure Machine Learning supporta PyTorch, TensorFlow e distribuito basato su MPI.

## 2.1 Configurazione dei comandi per la Lettura e la Scrittura dei dati

Il set di dati CIFAR 10, un file compresso, viene scaricato da un URL pubblico. Il codice **read_write_data.py** che si trova nella cartella **src** esegue l'estrazione dei file utilizzando la **libreria tarfile**.

In [ ]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes


inputs = {
    "cifar_zip": Input(
        type=AssetTypes.URI_FILE,
        path="https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz",
    ),
}

outputs = {
    "cifar": Output(
        type=AssetTypes.URI_FOLDER,
        path=f"azureml://subscriptions/{subscription_id}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/workspaceblobstore/paths/CIFAR-10",
    )
}

job = command(
    code="./src",  
    command="python read_write_data.py --input_data ${{inputs.cifar_zip}} --output_folder ${{outputs.cifar}}",
    inputs=inputs,
    outputs=outputs,
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1",
    compute="cpu-cluster",
)

returned_job = ml_client.jobs.create_or_update(job)
returned_job.studio_url

In [ ]:
ml_client.jobs.stream(returned_job.name)

In [ ]:
print(returned_job.name)
print(returned_job.experiment_name)
print(returned_job.outputs.cifar)
print(returned_job.outputs.cifar.path)

## 2.2 Configurazioni dei comandi per il Training Distribuito utilizzando Pytorch

In [ ]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes

inputs = {
    "cifar": Input(
        type=AssetTypes.URI_FOLDER, path=returned_job.outputs.cifar.path
    ),  
    "epoch": 10,
    "batchsize": 64,
    "workers": 2,
    "lr": 0.01,
    "momen": 0.9,
    "prtfreq": 200,
    "output": "./outputs",
}

job = command(
    code="./src",  
    command="python train.py --data-dir ${{inputs.cifar}} --epochs ${{inputs.epoch}} --batch-size ${{inputs.batchsize}} --workers ${{inputs.workers}} --learning-rate ${{inputs.lr}} --momentum ${{inputs.momen}} --print-freq ${{inputs.prtfreq}} --model-dir ${{inputs.output}}",
    inputs=inputs,
    environment="azureml:AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu:6",
    compute="gpu-cluster",  
    instance_count=2, 
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
)

# 3. Update del lavoro

In [ ]:
ml_client.jobs.create_or_update(job)